In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from metadata import *
from utilities2015 import *

Setting environment for Local Macbook Pro


In [12]:
import re

In [39]:
# Rename filenames using my stack name

new_stack_name = 'ChatCryoJane201710'

# input_dir = '/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_thumbnails_Neurotrace/'

# for fn in os.listdir(input_dir):
#     s = re.search("(.*?)_(slide.*?.tif)", fn)
#     the_rest = s.groups()[1]
#     new_fn = new_stack_name + '_' + the_rest
#     os.rename(os.path.join(input_dir, fn), os.path.join(input_dir, new_fn))
    
input_dir = '/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_thumbnails_Chat'

for fn in os.listdir(input_dir):
    s = re.search("(.*?)_(slide.*?.tif)", fn)
    the_rest = s.groups()[1]
    new_fn = new_stack_name + '_' + the_rest
    os.rename(os.path.join(input_dir, fn), os.path.join(input_dir, new_fn))

In [46]:
# Sort filenames

fn_to_properties = {}
for fn in os.listdir(input_dir):
    s = re.search("ChatCryoJane201710_slide([0-9]*)-S([0-9])-(.*?).tif", fn)
    slide_idx, section_idx, channel = s.groups()
    slide_idx = int(slide_idx)
    section_idx = int(section_idx)
    fn_to_properties[fn] = (slide_idx, section_idx, channel)
    
sorted_filenames = [os.path.splitext(f)[0] for p,f in sorted([(prop, fn) for fn, prop in fn_to_properties.items()])]

In [56]:
with open('/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_sorted_filenames.txt', 'w') as f:
    for overall_section_idx, fn in enumerate(sorted_filenames):
        f.write(fn + ' ' + str(overall_section_idx + 1) + '\n')

In [47]:
stack = 'ChatCryoJane201710'
input_dir = '/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_thumbnails_Neurotrace/'
output_dir = create_if_not_exists(os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output'))
filename_pairs = \
[{'prev_fn': sorted_filenames[i-1], 'curr_fn': sorted_filenames[i]} 
                             for i in range(1, len(sorted_filenames))]
fmt = 'tif'
suffix = 'thumbnail'

In [48]:
filename_pairs

[{'curr_fn': 'ChatCryoJane201710_slide1-S2-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide1-S1-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide1-S3-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide1-S2-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide2-S1-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide1-S3-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide2-S2-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide2-S1-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide2-S3-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide2-S2-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide3-S1-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide2-S3-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide3-S2-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide3-S1-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide3-S3-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide3-S2-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide4-S1-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide3-S3-Ntb'},
 {'curr_fn': 'ChatCryoJane201710_slide4-S2-Ntb',
  'prev_fn': 'ChatCryoJane201710_slide4-S1-Ntb'},
 {'curr_fn

In [53]:
parameter_dir = os.path.join(os.environ['REPO_DIR'], 'preprocess', 'parameters')

rg_param_rigid = os.path.join(parameter_dir, "Parameters_Rigid.txt")
rg_param_mutualinfo = os.path.join(parameter_dir, "Parameters_Rigid_MutualInfo.txt")

if stack in all_alt_nissl_ntb_stacks or stack in all_alt_nissl_tracing_stacks:
	rg_param = rg_param_mutualinfo
else:
	rg_param = rg_param_rigid

failed_pairs = []

for fn_pair in filename_pairs[40:41]:
	prev_fn = fn_pair['prev_fn']
	curr_fn = fn_pair['curr_fn']

	output_subdir = os.path.join(output_dir, curr_fn + '_to_' + prev_fn)

	if os.path.exists(output_subdir) and 'TransformParameters.0.txt' in os.listdir(output_subdir):
		sys.stderr.write('Result for aligning %s to %s already exists.\n' % (curr_fn, prev_fn))
		continue

	execute_command('rm -rf \"%s\"' % output_subdir)
	create_if_not_exists(output_subdir)

	ret = execute_command('%(elastix_bin)s -f \"%(fixed_fn)s\" -m \"%(moving_fn)s\" -out \"%(output_subdir)s\" -p \"%(rg_param)s\"' % \
			{'elastix_bin': ELASTIX_BIN,
			'rg_param': rg_param,
			'output_subdir': output_subdir,
			'fixed_fn': os.path.join(input_dir, prev_fn + '.' + fmt),
			'moving_fn': os.path.join(input_dir, curr_fn + '.' + fmt)
			})

	if ret == 1:
		# sys.stderr.write(prev_fn + ' vs. ' + curr_fn + ' failed.\n')
		failed_pairs.append((prev_fn, curr_fn))

import subprocess
hostname = subprocess.check_output("hostname", shell=True).strip()

if len(failed_pairs) > 0:
    with open(os.path.join(output_dir, '%s_failed_pairs_%s.txt' % (stack, hostname.split('.')[0])), 'w') as f:
        for pf, cf in failed_pairs:
            f.write(pf + ' ' + cf + '\n')

rm -rf "/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_elastix_output/ChatCryoJane201710_slide14-S3-Ntb_to_ChatCryoJane201710_slide14-S2-Ntb"
elastix -f "/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_thumbnails_Neurotrace/ChatCryoJane201710_slide14-S2-Ntb.tif" -m "/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_thumbnails_Neurotrace/ChatCryoJane201710_slide14-S3-Ntb.tif" -out "/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_elastix_output/ChatCryoJane201710_slide14-S3-Ntb_to_ChatCryoJane201710_slide14-S2-Ntb" -p "/home/yuncong/Brain/preprocess/parameters/Parameters_Rigid.txt"


In [ ]:
transform = np.reshape(map(np.float, args.transform.split(',')), (3,3))
background_color = args.background_color
sys.stderr.write("Background color: %s\n" % background_color)
suffix = args.suffix

x = args.x
y = args.y
w = args.w
h = args.h

if suffix == 'thumbnail':
    scale_factor = 1
elif suffix == 'lossy':
    scale_factor = 16
elif suffix == 'lossless':
    scale_factor = 32

T2 = transform.copy()
T2[:2,2] = transform[:2, 2] * scale_factor
T = np.linalg.inv(T2)

create_parent_dir_if_not_exists(output_fp)

execute_command("convert \"%(input_fp)s\" -virtual-pixel background -background %(bg_color)s +distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' -crop %(w)sx%(h)s%(x)s%(y)s\! -flatten -compress lzw \"%(output_fp)s\"" % \
                {'sx':T[0,0],
     'sy':T[1,1],
     'rx':T[1,0],
     'ry':T[0,1],
     'tx':T[0,2],
     'ty':T[1,2],
     'input_fp': input_fp,
     'output_fp': output_fp,
     'x': '+' + str(x * scale_factor) if int(x) >= 0 else str(x * scale_factor),
     'y': '+' + str(y * scale_factor) if int(y) >= 0 else str(y * scale_factor),
     'w': str(w * scale_factor),
     'h': str(h * scale_factor),
     'bg_color': background_color
    })